# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.


Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [ ]:
#Step 1 Access Internet
#importing selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys


#open chrome
nav = webdriver.Chrome()

#access site
nav.get('https://www.google.com')

#search for dollar terms
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação Dolar')
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#capture the information needed
cot_dolar = nav.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cot_dolar)

nav.get('https://www.google.com')
#finding element for google search, make the search and enter for euro
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação Euro')
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#capture the information needed
cot_euro = nav.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cot_euro)

#open browser for another site
nav.get('https://www.melhorcambio.com/ouro-hoje')
cot_ouro = nav.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cot_ouro = cot_ouro.replace(',', '.')
print(cot_ouro)

nav.quit()


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [ ]:
#importing the product list
import pandas as pd

df = pd.read_excel('Produtos.xlsx')
display(df)

- Atualizando os preços e o cálculo do Preço Final

In [39]:
#update the df

#update the value of Dollar, Euro and Gold, with the scraping data from the cell above
df.loc[df["Moeda"] == 'Dólar', 'Cotação'] = float(cot_dolar)
df.loc[df["Moeda"] == 'Euro', 'Cotação'] = float(cot_euro)
df.loc[df["Moeda"] == 'Ouro', 'Cotação'] = float(cot_ouro)

#updating the final price
df['Preço Base Reais'] = df['Preço Base Original'] * df['Cotação']
df['Preço Final'] = df['Preço Base Reais'] * df['Margem']

df['Preço Final'] = df['Preço Base Reais'] * df['Margem']
df['Preço Final'] = df['Preço Final'].map('{:.2f}'.format)
                                                            

display(df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.254000,5253.947460,1.40,7355.53
1,Carro Renault,4500.00,Euro,6.164702,27741.159405,2.00,55482.32
2,Notebook Dell,899.99,Dólar,5.254000,4728.547460,1.70,8038.53
3,IPhone,799.00,Dólar,5.254000,4197.946000,1.70,7136.51
4,Carro Fiat,3000.00,Euro,6.164702,18494.106270,1.90,35138.80
5,Celular Xiaomi,480.48,Dólar,5.254000,2524.441920,2.00,5048.88
6,Joia 20g,20.00,Ouro,296.310000,5926.200000,1.15,6815.13


### Agora vamos exportar a nova base de preços atualizada

In [40]:
df.to_excel('Produtos Novos.xlsx', index=False)